In [1]:
import numpy as np
import pandas as pd
import sklearn as skl

In [32]:
train = pd.read_json("data/music_200.json", lines=True)
test = pd.read_json("data/music_test_200.json", lines=True)
train.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,1384719342,"[0, 0]",5,"Not much to write about here, but it does exac...","02 28, 2014",A2IBPI20UZIR0U,"cassandra tu ""Yeah, well, that's just like, u...",good,1393545600
1,1384719342,"[13, 14]",5,The product does exactly as it should and is q...,"03 16, 2013",A14VAT5EAX3D9S,Jake,Jake,1363392000
2,1384719342,"[1, 1]",5,The primary job of this device is to block the...,"08 28, 2013",A195EZSQDW3E21,"Rick Bennette ""Rick Bennette""",It Does The Job Well,1377648000
3,1384719342,"[0, 0]",5,Nice windscreen protects my MXL mic and preven...,"02 14, 2014",A2C00NNG1ZQQG2,"RustyBill ""Sunday Rocker""",GOOD WINDSCREEN FOR THE MONEY,1392336000
4,1384719342,"[0, 0]",5,This pop filter is great. It looks and perform...,"02 21, 2014",A94QU4C90B1AX,SEAN MASLANKA,No more pops when I record my vocals.,1392940800


In [33]:
train = train.drop(['asin', 'helpful', 'reviewTime', 'reviewerID', 'reviewerName', 'summary', 'unixReviewTime'], axis=1)
test = test.drop(['asin', 'helpful', 'reviewTime', 'reviewerID', 'reviewerName', 'summary', 'unixReviewTime'], axis=1)
train.head()

,overall,reviewText
0,5,"Not much to write about here, but it does exac..."
1,5,The product does exactly as it should and is q...
2,5,The primary job of this device is to block the...
3,5,Nice windscreen protects my MXL mic and preven...
4,5,This pop filter is great. It looks and perform...


In [40]:
# train.assign(good = lambda g: g.overall >= 4)
# train.assign(good = lambda g: (1, 0)[g.overall >= 4])
train2 = train.assign(good = lambda g: g.overall >= 4)
train.assign(good = train2['good'].apply(lambda g: 1 if g else 0))

,overall,reviewText,good
0,5,"Not much to write about here, but it does exac...",1
1,5,The product does exactly as it should and is q...,1
2,5,The primary job of this device is to block the...,1
3,5,Nice windscreen protects my MXL mic and preven...,1
4,5,This pop filter is great. It looks and perform...,1
5,5,So good that I bought another one. Love the h...,1
6,5,"I have used monster cables for years, and with...",1
7,3,I now use this cable to run from the output of...,0
8,5,Perfect for my Epiphone Sheraton II. Monster ...,1
9,5,Monster makes the best cables and a lifetime w...,1


In [16]:
def rm_stopwords_punctuation(text):
    with read("stopwords.json") as stopwords:
        for word in stopwords:
            text.replace(word, "")
    # TODO: remove punctuation
    return text

,helpful,overall,reviewText
3,"[0, 0]",4,... but... they're the cheesiest looking leath...
18,"[0, 0]",4,I have about quit using picks now. But I need...
21,"[0, 0]",4,Couldn't be happier than playing with a variet...
27,"[1, 1]",4,If you are picky these are all eye candy... ni...
28,"[0, 0]",4,Throw this right in my little storage bag I us...
30,"[0, 0]",4,"Solid, and super easy to use. The best for the..."
41,"[1, 1]",4,I bought this along with a new guitar for my d...
43,"[2, 2]",4,"Luv it, I had version #1 and this amp isnt muc..."
47,"[0, 0]",4,"works as advertised, good construction, folds ..."
56,"[2, 2]",4,I like the fact is foldable and can be put awa...
